In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, preprocessing

In [ ]:
train_dir = '/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Train'
valid_dir = '/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Validation'
test_dir = '/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Test'

In [ ]:
num_classes = 2

train_datagen = preprocessing.image.ImageDataGenerator(rescale=1./255, 
rotation_range=40, 
width_shift_range=0.2,  
height_shift_range=0.2, 
zoom_range=0.2, 
horizontal_flip=True, 
fill_mode='nearest')

val_test_datagen = preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(directory= train_dir, target_size=(150,150))
val_generator = val_test_datagen.flow_from_directory(directory= valid_dir, target_size=(150,150))
test_generator = val_test_datagen.flow_from_directory(directory= test_dir, target_size=(150,150))

In [ ]:
# res_model = tf.keras.applications.resnet50.ResNet50(include_top= False, weights='imagenet',input_shape=(150,150,3))

# for layer in res_model.layers:
#     layer.trainable = False

In [ ]:
# model = tf.keras.Sequential()
# model.add(res_model)
# model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Dense(2, activation='sigmoid'))

# model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001), loss="categorical_crossentropy",metrics ="accuracy")
# model.summary()

In [ ]:
# history = model.fit(train_generator, steps_per_epoch= len(train_generator)//32, validation_data= val_generator,
#                     validation_steps= len(val_generator)//32, verbose=1, epochs=20)

In [ ]:
vgg_model = tf.keras.applications.vgg19.VGG19(weights='imagenet',include_top=False,input_shape=(150,150,3))

for layer in vgg_model.layers:
    layer.trainable = False

In [ ]:
best_model = tf.keras.callbacks.ModelCheckpoint('/kaggle/working/vgg19best.h5',save_best_only= True, monitor='val_accuracy', save_freq='epoch')

In [ ]:
initial_learning_rate = 0.01

model = tf.keras.Sequential()
model.add(vgg_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(2, activation='sigmoid'))

model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate= tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10,
    decay_rate=0.9
), ), loss="categorical_crossentropy",metrics ="accuracy")
model.summary()

In [ ]:
history = model.fit(train_generator, steps_per_epoch= len(train_generator)//32, validation_data= val_generator,
                    validation_steps= len(val_generator)//32, verbose=1, epochs=20, callbacks=[best_model])

In [ ]:
model.evaluate(test_generator)

In [ ]:
model.save('/kaggle/working/vgg19modelbetter.h5')